In [ ]:
import json
import os
import time
from typing import Any

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd

from sklearn.metrics import f1_score   
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

In [ ]:
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sys.path.insert(0, '/content/drive/MyDrive/ColabNotebooks')

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#import RoadImageReader

In [ ]:
from  drive.MyDrive.ColabNotebooks.RoadImageReader import RoadImageReader

In [ ]:
from drive.MyDrive.ColabNotebooks.GaborNN5layer import GaborNN5layer

In [ ]:
os.getcwd()

os.path.join("drive/MyDrive/Colab Notebooks/data", "train")

'drive/MyDrive/Colab Notebooks/data/train'

In [ ]:
transform = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [ ]:
train_set = RoadImageReader(
        root_dir=os.path.join("drive/MyDrive/ColabNotebooks", "train"), transform=transform
    )
test_set = RoadImageReader(
        root_dir=os.path.join("drive/MyDrive/ColabNotebooks", "test"), transform=transform
    )

In [ ]:
train = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
test = DataLoader(test_set, batch_size=128, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
net = GaborNN5layer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1,cycle_momentum=False)

In [ ]:
epochs=50

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []

data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
def setDataDict(model,epoch,trainacc,trainloss,testacc,testloss,time,trpre,trre,trf1,tepre,tere,tef1,trainauc,testauc):
    data_dict['MODEL'].append(model)
    data_dict['EPOCH'].append(epoch)
    data_dict['TRAINACC'].append(trainacc)
    data_dict['TRAINLOSS'].append(trainloss)
    data_dict['TESTACC'].append(testacc)
    data_dict['TESTLOSS'].append(testloss)
    data_dict['TIME'].append(time)
    data_dict['TRAINPRECISION'].append(trpre)
    data_dict['TRAINRECALL'].append(trre)
    data_dict['TRAINF1SCORE'].append(trf1)
    data_dict['TESTPRECISION'].append(tepre)
    data_dict['TESTRECALL'].append(tere)
    data_dict['TESTF1SCORE'].append(tef1)
    data_dict['TRAINAUC'].append(trainauc)
    data_dict['TESTAUC'].append(testauc)
    

In [ ]:
def trainingData(net,criterion,optimizer,epochs,modelName):
    for epoch in range(epochs):

        epochTime=time.perf_counter()
        running_loss = 0.0
        correct = 0
        trainrecall=[]
        trainprecision=[]
        testrecall=[]
        testprecision=[]
        trainauc=[]
        testauc=[]
        label=[]
        net.train()
        start = time.perf_counter()
        for data in train:
            # get the inputs
            inputs, labels = data["image"], data["target"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            #scheduler.step()
            pred = outputs.max(1, keepdim=True)[1].to("cpu")

            correct += pred.eq(labels.view_as(pred)).sum().item()
            trainprecision.append(precision_score(labels.data, pred))
            trainrecall.append(recall_score(labels.data, pred))
            trainauc.append(roc_auc_score(labels.data, pred))
            
            
            # print statistics
            running_loss += loss.item()
        finish = time.perf_counter()

        trainPrecisionScore=sum(trainprecision)/len(trainprecision)
        trainRecallScore=sum(trainrecall)/len(trainrecall)
        trainF1score=(2*(trainPrecisionScore*trainRecallScore))/(trainPrecisionScore+trainRecallScore)
        trainAUC=sum(trainauc)/len(trainauc)
        

        time_per_image_train.append((finish - start) / len(train_set))
        print(
            "[%d] train_acc: %.3f train_loss: %.3f"
            % (epoch + 1, correct / len(train_set), running_loss / len(train_set))
        )
        one_layer_gnet_acc_train.append(correct / len(train_set))
        train_loss=running_loss / len(train_set)
        train_acc=correct / len(train_set)
        running_loss = 0.0
        correct = 0
        start = time.perf_counter()
        with torch.no_grad():
            net.eval()
            for data in test:
                # get the inputs
                inputs, labels = data["image"], data["target"]

                # forward + backward + optimize
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                pred = outputs.max(1, keepdim=True)[1].to("cpu")
                correct += pred.eq(labels.view_as(pred)).sum().item()
                testprecision.append(precision_score(labels.data, pred))
                testrecall.append(recall_score(labels.data, pred))
                testauc.append(roc_auc_score(labels.data, pred))
                running_loss += loss.item()
        finish = time.perf_counter()
        testPrecisionScore=sum(testprecision)/len(testprecision)
        testRecallScore=sum(testrecall)/len(testrecall)
        testF1score=(2*(testPrecisionScore*testRecallScore))/(testPrecisionScore+testRecallScore)
        testAUC=sum(testauc)/len(testauc)
        time_per_image_test.append((finish - start) / len(test_set))
        print(
            "[%d] test_acc: %.3f test_loss: %.3f"
            % (epoch + 1, correct / len(test_set), running_loss / len(test_set))
        )
        one_layer_gnet_acc_test.append(correct / len(test_set))
        epochTime=time.perf_counter()-epochTime
        setDataDict(modelName,epoch+1,train_acc,train_loss,correct / len(test_set), running_loss / len(test_set),epochTime,trainPrecisionScore,trainRecallScore,trainF1score,testPrecisionScore,testRecallScore,testF1score,trainAUC,testAUC)
        torch.save(net, "drive/MyDrive/ColabNotebooks/"+modelName+"/"+modelName+str(epoch+1)+".pt")
        print("Time Taken in Epoch "+ str(epoch+1)+"  Seconds: "+str(epochTime))

    print("Finished Training")

In [ ]:
trainingData(net,criterion,optimizer,epochs,'GaborNN5layer')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[1] train_acc: 0.538 train_loss: 0.005
[1] test_acc: 0.619 test_loss: 0.005
Time Taken in Epoch 1  Seconds: 56.67322315100046
[2] train_acc: 0.620 train_loss: 0.005
[2] test_acc: 0.648 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 56.87181915900055
[3] train_acc: 0.646 train_loss: 0.005
[3] test_acc: 0.697 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 56.892392126999766
[4] train_acc: 0.671 train_loss: 0.005
[4] test_acc: 0.703 test_loss: 0.004
Time Taken in Epoch 4  Seconds: 57.48751405799976
[5] train_acc: 0.681 train_loss: 0.005
[5] test_acc: 0.717 test_loss: 0.004
Time Taken in Epoch 5  Seconds: 57.691446448000534
[6] train_acc: 0.695 train_loss: 0.004
[6] test_acc: 0.725 test_loss: 0.004
Time Taken in Epoch 6  Seconds: 56.40978175800046
[7] train_acc: 0.712 train_loss: 0.004
[7] test_acc: 0.743 test_loss: 0.004
Time Taken in Epoch 7  Seconds: 56.87533288400027
[8] train_acc: 0.719 train_loss: 0.004
[8] test_acc: 0.744 test_loss: 0.004
Time Taken in Epoch 8  Seconds: 56.76775

In [ ]:
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
torch.save(net.state_dict(), "gabornn.pt")
torch.save(net, "drive/MyDrive/ColabNotebooks/GaborNN5layer.pt")

In [ ]:
#model = torch.load('GaborNN5layer.pt')
#model.eval()

In [ ]:
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
epochs=50

In [ ]:
from  drive.MyDrive.ColabNotebooks.GaborNN8Layer import GaborNN8Layer

In [ ]:
net =GaborNN8Layer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=50

In [ ]:
trainingData(net,criterion,optimizer,epochs,'GaborNN7Layer')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1] train_acc: 0.506 train_loss: 0.010
[1] test_acc: 0.496 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 23.35896753599991
[2] train_acc: 0.539 train_loss: 0.006
[2] test_acc: 0.507 test_loss: 0.006
Time Taken in Epoch 2  Seconds: 23.292081178000444
[3] train_acc: 0.567 train_loss: 0.005
[3] test_acc: 0.565 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 23.536812636999457
[4] train_acc: 0.590 train_loss: 0.005
[4] test_acc: 0.610 test_loss: 0.005
Time Taken in Epoch 4  Seconds: 23.392191795001054
[5] train_acc: 0.612 train_loss: 0.005
[5] test_acc: 0.634 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 23.337387765999665
[6] train_acc: 0.632 train_loss: 0.005
[6] test_acc: 0.651 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 23.32094401700124
[7] train_acc: 0.649 train_loss: 0.005
[7] test_acc: 0.662 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 23.45062652100023
[8] train_acc: 0.653 train_loss: 0.005
[8] test_acc: 0.666 test_loss: 0.005
Time Taken in Epoch 8  Seconds: 23.365

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/GaborNN7Layer.pt")

In [ ]:
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
from  drive.MyDrive.ColabNotebooks.ModelNN import SimpleNN

In [ ]:
net =SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=50

In [ ]:
trainingData(net,criterion,optimizer,epochs,'CNN')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[1] train_acc: 0.552 train_loss: 0.005
[1] test_acc: 0.525 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 22.319198885999867
[2] train_acc: 0.569 train_loss: 0.005
[2] test_acc: 0.538 test_loss: 0.006
Time Taken in Epoch 2  Seconds: 21.454054693000217
[3] train_acc: 0.569 train_loss: 0.005
[3] test_acc: 0.517 test_loss: 0.006
Time Taken in Epoch 3  Seconds: 21.386101146999863
[4] train_acc: 0.577 train_loss: 0.005
[4] test_acc: 0.533 test_loss: 0.006
Time Taken in Epoch 4  Seconds: 21.597134968998944
[5] train_acc: 0.579 train_loss: 0.005
[5] test_acc: 0.534 test_loss: 0.006
Time Taken in Epoch 5  Seconds: 21.530588296000133
[6] train_acc: 0.582 train_loss: 0.005
[6] test_acc: 0.549 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 21.35657682099918
[7] train_acc: 0.585 train_loss: 0.005
[7] test_acc: 0.563 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 21.635178735999943
[8] train_acc: 0.595 train_loss: 0.005
[8] test_acc: 0.561 test_loss: 0.005
Time Taken in Epoch 8  Seconds: 21.5

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/CNN.pt")

In [ ]:
df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
data_dict

{'EPOCH': [],
 'MODEL': [],
 'TESTACC': [],
 'TESTAUC': [],
 'TESTF1SCORE': [],
 'TESTLOSS': [],
 'TESTPRECISION': [],
 'TESTRECALL': [],
 'TIME': [],
 'TRAINACC': [],
 'TRAINAUC': [],
 'TRAINF1SCORE': [],
 'TRAINLOSS': [],
 'TRAINPRECISION': [],
 'TRAINRECALL': []}

In [ ]:
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
from  drive.MyDrive.ColabNotebooks.GaborNNWithBN import GaborNNWithBN

In [ ]:
net =GaborNNWithBN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=50

In [ ]:
trainingData(net,criterion,optimizer,epochs,'GaborNNWithBN')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1] train_acc: 0.535 train_loss: 0.007


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[1] test_acc: 0.500 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 22.37210747899917
[2] train_acc: 0.562 train_loss: 0.005
[2] test_acc: 0.504 test_loss: 0.006
Time Taken in Epoch 2  Seconds: 22.935315369999444
[3] train_acc: 0.555 train_loss: 0.005
[3] test_acc: 0.513 test_loss: 0.006
Time Taken in Epoch 3  Seconds: 22.550061678999555
[4] train_acc: 0.564 train_loss: 0.005
[4] test_acc: 0.538 test_loss: 0.006
Time Taken in Epoch 4  Seconds: 22.70467629599989
[5] train_acc: 0.577 train_loss: 0.005
[5] test_acc: 0.566 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 22.617348673999004
[6] train_acc: 0.585 train_loss: 0.005
[6] test_acc: 0.600 test_loss: 0.006
Time Taken in Epoch 6  Seconds: 22.547977751999497
[7] train_acc: 0.609 train_loss: 0.005
[7] test_acc: 0.629 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 23.96469417499975
[8] train_acc: 0.621 train_loss: 0.005
[8] test_acc: 0.602 test_loss: 0.005
Time Taken in Epoch 8  Seconds: 22.92119806499977
[9] train_acc: 0.612 train_

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/GaborNNWithBN.pt")

In [ ]:
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
trainUncrackedDir='drive/MyDrive/ColabNotebooks/train/uncracked/'
trainCrackedDir='drive/MyDrive/ColabNotebooks/train/cracked/'

uncrackedList=os.listdir(trainUncrackedDir)
crackedList=os.listdir(trainCrackedDir)

In [ ]:
len(crackedList)

7256

In [ ]:
len(uncrackedList)

7256

In [ ]:
from  drive.MyDrive.ColabNotebooks.GaborNN import GaborNN

In [ ]:
net =GaborNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=50

In [ ]:
trainingData(net,criterion,optimizer,epochs,'GaborNN')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[1] train_acc: 0.558 train_loss: 0.005


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[1] test_acc: 0.499 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 27.146876114000406


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/

[2] train_acc: 0.560 train_loss: 0.005
[2] test_acc: 0.509 test_loss: 0.006
Time Taken in Epoch 2  Seconds: 23.035468418000164


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[3] train_acc: 0.558 train_loss: 0.005
[3] test_acc: 0.517 test_loss: 0.006
Time Taken in Epoch 3  Seconds: 23.014894926000125
[4] train_acc: 0.562 train_loss: 0.005
[4] test_acc: 0.532 test_loss: 0.006
Time Taken in Epoch 4  Seconds: 22.857374137998704
[5] train_acc: 0.576 train_loss: 0.005
[5] test_acc: 0.539 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 22.95597639100015
[6] train_acc: 0.576 train_loss: 0.005
[6] test_acc: 0.575 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 22.858789791000163
[7] train_acc: 0.593 train_loss: 0.005
[7] test_acc: 0.562 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 23.036646774000474
[8] train_acc: 0.601 train_loss: 0.005
[8] test_acc: 0.604 test_loss: 0.005
Time Taken in Epoch 8  Seconds: 22.98915760499949
[9] train_acc: 0.608 train_loss: 0.005
[9] test_acc: 0.607 test_loss: 0.005
Time Taken in Epoch 9  Seconds: 22.83334946099967
[10] train_acc: 0.624 train_loss: 0.005
[10] test_acc: 0.616 test_loss: 0.005
Time Taken in Epoch 10  Seconds: 22.

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/GaborNN.pt")

In [ ]:
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
from  drive.MyDrive.ColabNotebooks.GaborNN8LayerWithoutBN import GaborNN8LayerWithoutBN

In [ ]:
net =GaborNN8LayerWithoutBN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=50

In [ ]:
trainingData(net,criterion,optimizer,epochs,'GaborNN8LayerWithoutBN')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[1] train_acc: 0.505 train_loss: 0.006
[1] test_acc: 0.522 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 56.658607027999096
[2] train_acc: 0.513 train_loss: 0.005
[2] test_acc: 0.508 test_loss: 0.006
Time Taken in Epoch 2  Seconds: 57.50738821400046
[3] train_acc: 0.497 train_loss: 0.005
[3] test_acc: 0.502 test_loss: 0.006
Time Taken in Epoch 3  Seconds: 56.85106341699975


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[4] train_acc: 0.501 train_loss: 0.005
[4] test_acc: 0.513 test_loss: 0.006
Time Taken in Epoch 4  Seconds: 57.02587732599932


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

...

[48] test_acc: 0.499 test_loss: 0.006
Time Taken in Epoch 48  Seconds: 52.143647692999366


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[49] train_acc: 0.508 train_loss: 0.005
[49] test_acc: 0.501 test_loss: 0.006
Time Taken in Epoch 49  Seconds: 52.378829566001514


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

[50] train_acc: 0.499 train_loss: 0.005
[50] test_acc: 0.509 test_loss: 0.006
Time Taken in Epoch 50  Seconds: 52.69229347200053
Finished Training


In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/GaborNN8LayerWithoutBN.pt")

In [ ]:
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }

In [ ]:
model = torch.load('drive/MyDrive/ColabNotebooks/GaborNN8LayerWithoutBN.pt')
model.eval()

GaborNN8LayerWithoutBN(
  (g1): GaborConv2dLayer(
    (conv_layer): Conv2d(3, 32, kernel_size=(15, 15), stride=(1, 1), bias=False)
  )
  (c1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (c2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (c3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
  (c4): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
  (c5): Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1))
  (c6): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=50176, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=2, bias=True)
)

In [ ]:
sum(dict((p.data_ptr(), p.numel()) for p in model.parameters()).values())

53259206

In [ ]:
from  drive.MyDrive.ColabNotebooks.GaborNN8Layer import GaborNN8Layer

net =GaborNN8Layer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=50

In [ ]:
trainingData(net,criterion,optimizer,epochs,'GaborNN7Layer')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1] train_acc: 0.525 train_loss: 0.007
[1] test_acc: 0.561 test_loss: 0.005
Time Taken in Epoch 1  Seconds: 52.36908278900046
[2] train_acc: 0.603 train_loss: 0.005
[2] test_acc: 0.657 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 52.67454952799926
[3] train_acc: 0.636 train_loss: 0.005
[3] test_acc: 0.687 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 52.610508206000304
[4] train_acc: 0.651 train_loss: 0.005
[4] test_acc: 0.696 test_loss: 0.005
Time Taken in Epoch 4  Seconds: 52.37760345499919
[5] train_acc: 0.664 train_loss: 0.005
[5] test_acc: 0.713 test_loss: 0.004
Time Taken in Epoch 5  Seconds: 53.01854097700016
[6] train_acc: 0.676 train_loss: 0.005
[6] test_acc: 0.722 test_loss: 0.004
Time Taken in Epoch 6  Seconds: 52.58875772399915
[7] train_acc: 0.687 train_loss: 0.005
[7] test_acc: 0.724 test_loss: 0.004
Time Taken in Epoch 7  Seconds: 52.71126250599991
[8] train_acc: 0.701 train_loss: 0.004
[8] test_acc: 0.729 test_loss: 0.004
Time Taken in Epoch 8  Seconds: 52.943217

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/GaborNN7Layer.pt")
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }


In [ ]:
def trainingDataForTesting(net,criterion,optimizer,epochs,modelName):
    for epoch in range(epochs):

        epochTime=time.perf_counter()
        running_loss = 0.0
        correct = 0
        trainrecall=[]
        trainprecision=[]
        testrecall=[]
        testprecision=[]
        trainauc=[]
        testauc=[]
        label=[]
        net.train()
        start = time.perf_counter()
        for data in train:
            # get the inputs
            inputs, labels = data["image"], data["target"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            #scheduler.step()
            pred = outputs.max(1, keepdim=True)[1].to("cpu")

            correct += pred.eq(labels.view_as(pred)).sum().item()
            trainprecision.append(precision_score(labels.data, pred))
            trainrecall.append(recall_score(labels.data, pred))
            trainauc.append(roc_auc_score(labels.data, pred))
            
            
            # print statistics
            running_loss += loss.item()
        finish = time.perf_counter()

        trainPrecisionScore=sum(trainprecision)/len(trainprecision)
        trainRecallScore=sum(trainrecall)/len(trainrecall)
        trainF1score=(2*(trainPrecisionScore*trainRecallScore))/(trainPrecisionScore+trainRecallScore)
        trainAUC=sum(trainauc)/len(trainauc)
        

        time_per_image_train.append((finish - start) / len(train_set))
        print(
            "[%d] train_acc: %.3f train_loss: %.3f"
            % (epoch + 1, correct / len(train_set), running_loss / len(train_set))
        )
        one_layer_gnet_acc_train.append(correct / len(train_set))
        train_loss=running_loss / len(train_set)
        train_acc=correct / len(train_set)
        running_loss = 0.0
        correct = 0
        start = time.perf_counter()
        with torch.no_grad():
            net.eval()
            for data in test:
                # get the inputs
                inputs, labels = data["image"], data["target"]

                # forward + backward + optimize
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                pred = outputs.max(1, keepdim=True)[1].to("cpu")
                correct += pred.eq(labels.view_as(pred)).sum().item()
                testprecision.append(precision_score(labels.data, pred))
                testrecall.append(recall_score(labels.data, pred))
                testauc.append(roc_auc_score(labels.data, pred))
                running_loss += loss.item()
        finish = time.perf_counter()
        testPrecisionScore=sum(testprecision)/len(testprecision)
        testRecallScore=sum(testrecall)/len(testrecall)
        testF1score=(2*(testPrecisionScore*testRecallScore))/(testPrecisionScore+testRecallScore)
        testAUC=sum(testauc)/len(testauc)
        time_per_image_test.append((finish - start) / len(test_set))
        print(
            "[%d] test_acc: %.3f test_loss: %.3f"
            % (epoch + 1, correct / len(test_set), running_loss / len(test_set))
        )
        one_layer_gnet_acc_test.append(correct / len(test_set))
        epochTime=time.perf_counter()-epochTime
        #setDataDict(modelName,epoch+1,train_acc,train_loss,correct / len(test_set), running_loss / len(test_set),epochTime,trainPrecisionScore,trainRecallScore,trainF1score,testPrecisionScore,testRecallScore,testF1score,trainAUC,testAUC)
        #torch.save(net, "drive/MyDrive/ColabNotebooks/"+modelName+"/"+modelName+str(epoch+1)+".pt")
        print("Time Taken in Epoch "+ str(epoch+1)+"  Seconds: "+str(epochTime))
        

    print("Finished Training")

In [ ]:
net =GaborNN8Layer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
weight_decay =1e-4
optimizer1 = optim.Adam(net.parameters(), lr=0.001)
optimizer2 = optim.Adam(net.parameters(), lr=0.001, weight_decay= weight_decay)


one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=70

In [ ]:
trainingData(net,criterion,optimizer2,epochs,'ProposedModel')

[1] train_acc: 0.548 train_loss: 0.006
[1] test_acc: 0.589 test_loss: 0.005
Time Taken in Epoch 1  Seconds: 56.88550325799952
[2] train_acc: 0.611 train_loss: 0.005
[2] test_acc: 0.657 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 54.37289404599869
[3] train_acc: 0.637 train_loss: 0.005
[3] test_acc: 0.665 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 53.360165031997894
[4] train_acc: 0.657 train_loss: 0.005
[4] test_acc: 0.700 test_loss: 0.004
Time Taken in Epoch 4  Seconds: 53.361548363998736
[5] train_acc: 0.671 train_loss: 0.005
[5] test_acc: 0.707 test_loss: 0.004
Time Taken in Epoch 5  Seconds: 53.20548644099836
[6] train_acc: 0.683 train_loss: 0.005
[6] test_acc: 0.723 test_loss: 0.004
Time Taken in Epoch 6  Seconds: 53.10254886899929
[7] train_acc: 0.699 train_loss: 0.004
[7] test_acc: 0.731 test_loss: 0.004
Time Taken in Epoch 7  Seconds: 53.51090952099912
[8] train_acc: 0.702 train_loss: 0.004
[8] test_acc: 0.733 test_loss: 0.004
Time Taken in Epoch 8  Seconds: 52.74279

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/ProposedModel.pt")
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }


In [ ]:
net =GaborNN8Layer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
weight_decay =1e-4
optimizer1 = optim.Adam(net.parameters(), lr=0.001)
optimizer2 = optim.Adam(net.parameters(), lr=0.001, weight_decay= weight_decay)


one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []

In [ ]:
net=torch.load( "drive/MyDrive/ColabNotebooks/ProposedModel.pt")

In [ ]:
def trainingDataStart(net,criterion,optimizer,epochs,modelName,startepoch):
    for epoch in range(epochs):
        epoch=startepoch+epoch
        epochTime=time.perf_counter()
        running_loss = 0.0
        correct = 0
        trainrecall=[]
        trainprecision=[]
        testrecall=[]
        testprecision=[]
        trainauc=[]
        testauc=[]
        label=[]
        net.train()
        start = time.perf_counter()
        for data in train:
            # get the inputs
            inputs, labels = data["image"], data["target"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            #scheduler.step()
            pred = outputs.max(1, keepdim=True)[1].to("cpu")

            correct += pred.eq(labels.view_as(pred)).sum().item()
            trainprecision.append(precision_score(labels.data, pred))
            trainrecall.append(recall_score(labels.data, pred))
            trainauc.append(roc_auc_score(labels.data, pred))
            
            
            # print statistics
            running_loss += loss.item()
        finish = time.perf_counter()

        trainPrecisionScore=sum(trainprecision)/len(trainprecision)
        trainRecallScore=sum(trainrecall)/len(trainrecall)
        trainF1score=(2*(trainPrecisionScore*trainRecallScore))/(trainPrecisionScore+trainRecallScore)
        trainAUC=sum(trainauc)/len(trainauc)
        

        time_per_image_train.append((finish - start) / len(train_set))
        print(
            "[%d] train_acc: %.3f train_loss: %.3f"
            % (epoch + 1, correct / len(train_set), running_loss / len(train_set))
        )
        one_layer_gnet_acc_train.append(correct / len(train_set))
        train_loss=running_loss / len(train_set)
        train_acc=correct / len(train_set)
        running_loss = 0.0
        correct = 0
        start = time.perf_counter()
        with torch.no_grad():
            net.eval()
            for data in test:
                # get the inputs
                inputs, labels = data["image"], data["target"]

                # forward + backward + optimize
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                pred = outputs.max(1, keepdim=True)[1].to("cpu")
                correct += pred.eq(labels.view_as(pred)).sum().item()
                testprecision.append(precision_score(labels.data, pred))
                testrecall.append(recall_score(labels.data, pred))
                testauc.append(roc_auc_score(labels.data, pred))
                running_loss += loss.item()
        finish = time.perf_counter()
        testPrecisionScore=sum(testprecision)/len(testprecision)
        testRecallScore=sum(testrecall)/len(testrecall)
        testF1score=(2*(testPrecisionScore*testRecallScore))/(testPrecisionScore+testRecallScore)
        testAUC=sum(testauc)/len(testauc)
        time_per_image_test.append((finish - start) / len(test_set))
        print(
            "[%d] test_acc: %.3f test_loss: %.3f"
            % (epoch + 1, correct / len(test_set), running_loss / len(test_set))
        )
        one_layer_gnet_acc_test.append(correct / len(test_set))
        epochTime=time.perf_counter()-epochTime
        setDataDict(modelName,epoch+1,train_acc,train_loss,correct / len(test_set), running_loss / len(test_set),epochTime,trainPrecisionScore,trainRecallScore,trainF1score,testPrecisionScore,testRecallScore,testF1score,trainAUC,testAUC)
        torch.save(net, "drive/MyDrive/ColabNotebooks/"+modelName+"/"+modelName+str(epoch+1)+".pt")
        print("Time Taken in Epoch "+ str(epoch+1)+"  Seconds: "+str(epochTime))

    print("Finished Training")

In [ ]:
trainingDataStart(net,criterion,optimizer2,50,'ProposedModel',50)

[51] train_acc: 0.867 train_loss: 0.002
[51] test_acc: 0.758 test_loss: 0.005
Time Taken in Epoch 51  Seconds: 56.03240458499931
[52] train_acc: 0.865 train_loss: 0.002
[52] test_acc: 0.758 test_loss: 0.005
Time Taken in Epoch 52  Seconds: 53.94871396700182
[53] train_acc: 0.863 train_loss: 0.002
[53] test_acc: 0.755 test_loss: 0.005
Time Taken in Epoch 53  Seconds: 53.90868387199953
[54] train_acc: 0.863 train_loss: 0.002
[54] test_acc: 0.753 test_loss: 0.005
Time Taken in Epoch 54  Seconds: 54.01672609700108
[55] train_acc: 0.864 train_loss: 0.002
[55] test_acc: 0.752 test_loss: 0.005
Time Taken in Epoch 55  Seconds: 53.42628690499987
[56] train_acc: 0.863 train_loss: 0.002
[56] test_acc: 0.761 test_loss: 0.005
Time Taken in Epoch 56  Seconds: 53.69374468100068
[57] train_acc: 0.865 train_loss: 0.002
[57] test_acc: 0.767 test_loss: 0.005
Time Taken in Epoch 57  Seconds: 53.59796742799881
[58] train_acc: 0.863 train_loss: 0.002
[58] test_acc: 0.749 test_loss: 0.005
Time Taken in Epoch

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/ProposedModel.pt")
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }


In [ ]:
trainingDataStart(net,criterion,optimizer2,200,'ProposedModel',100)

[101] train_acc: 0.867 train_loss: 0.002
[101] test_acc: 0.760 test_loss: 0.005
Time Taken in Epoch 101  Seconds: 53.85761162500421
[102] train_acc: 0.866 train_loss: 0.002
[102] test_acc: 0.759 test_loss: 0.005
Time Taken in Epoch 102  Seconds: 54.41061185500439
[103] train_acc: 0.865 train_loss: 0.002
[103] test_acc: 0.757 test_loss: 0.005
Time Taken in Epoch 103  Seconds: 54.03372499400575
[104] train_acc: 0.867 train_loss: 0.002
[104] test_acc: 0.749 test_loss: 0.006
Time Taken in Epoch 104  Seconds: 53.969519396996475
[105] train_acc: 0.861 train_loss: 0.002
[105] test_acc: 0.755 test_loss: 0.006
Time Taken in Epoch 105  Seconds: 54.06626269700064
[106] train_acc: 0.865 train_loss: 0.002
[106] test_acc: 0.755 test_loss: 0.005
Time Taken in Epoch 106  Seconds: 53.917435003997525
[107] train_acc: 0.864 train_loss: 0.002
[107] test_acc: 0.757 test_loss: 0.005
Time Taken in Epoch 107  Seconds: 54.087956630995905
[108] train_acc: 0.867 train_loss: 0.002
[108] test_acc: 0.758 test_loss:

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

[245] train_acc: 0.867 train_loss: 0.002


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

[245] test_acc: 0.755 test_loss: 0.006
Time Taken in Epoch 245  Seconds: 55.87539489099436


...

[260] train_acc: 0.868 train_loss: 0.002


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
Traceback (most recent call last):
  File "/usr/local/lib/python3

[260] test_acc: 0.759 test_loss: 0.005
Time Taken in Epoch 260  Seconds: 55.373391990004166


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40695145f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

KeyboardInterrupt: ignored

In [ ]:
torch.save(net, "drive/MyDrive/ColabNotebooks/ProposedModel.pt")
df=pd.read_csv("drive/MyDrive/ColabNotebooks/Result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("drive/MyDrive/ColabNotebooks/Result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],
             'TRAINAUC':[],
        'TESTAUC':[],
            }
